In [1]:
!pip install psycopg2-binary

     |████████████████████████████████| 2.9MB 4.7MB/s 


In [0]:
import psycopg2

In [3]:
dir(psycopg2)

['BINARY',
 'Binary',
 'DATETIME',
 'DataError',
 'DatabaseError',
 'Date',
 'DateFromTicks',
 'Error',
 'IntegrityError',
 'InterfaceError',
 'InternalError',
 'NUMBER',
 'NotSupportedError',
 'OperationalError',
 'ProgrammingError',
 'ROWID',
 'STRING',
 'Time',
 'TimeFromTicks',
 'Timestamp',
 'TimestampFromTicks',
 'Warning',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__libpq_version__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_connect',
 '_ext',
 '_json',
 '_psycopg',
 '_range',
 'apilevel',
 'compat',
 'connect',
 'errors',
 'extensions',
 'paramstyle',
 'threadsafety',
 'tz']

In [0]:
dbname = 'kqdffrqi'
user = 'kqdffrqi'
password = 'ZwZyobnB8PGRbabHyr9xB8gETfYfsuTX'
host = 'rajje.db.elephantsql.com'

In [0]:
pg_conn = psycopg2.connect(dbname=dbname,
                           user=user,
                           password=password,
                           host=host)

In [24]:
pg_conn

<connection object at 0x7efe695fa3f0; dsn: 'user=kqdffrqi password=xxx dbname=kqdffrqi host=rajje.db.elephantsql.com', closed: 0>

In [11]:
dir(pg_conn)

['DataError',
 'DatabaseError',
 'Error',
 'IntegrityError',
 'InterfaceError',
 'InternalError',
 'NotSupportedError',
 'OperationalError',
 'ProgrammingError',
 'Warning',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'async',
 'async_',
 'autocommit',
 'binary_types',
 'cancel',
 'close',
 'closed',
 'commit',
 'cursor',
 'cursor_factory',
 'deferrable',
 'dsn',
 'encoding',
 'fileno',
 'get_backend_pid',
 'get_dsn_parameters',
 'get_native_connection',
 'get_parameter_status',
 'get_transaction_status',
 'info',
 'isexecuting',
 'isolation_level',
 'lobject',
 'notices',
 'notifies',
 'pgconn_ptr',
 'poll',
 'protocol_version',
 'readonly',
 'reset',
 'rollback',
 'server_version

In [0]:
pg_curs = pg_conn.cursor()

In [20]:
help(pg_curs.execute)

Help on built-in function execute:

execute(...) method of psycopg2.extensions.cursor instance
    execute(query, vars=None) -- Execute query with bound vars.



In [0]:
create_table_statement = """
CREATE TABLE test_table (
  id        SERIAL PRIMARY KEY,
  name  varchar(40) NOT NULL,
  data    JSONB
);
"""

insert_statement = """
INSERT INTO test_table (name, data) VALUES
(
  'A row name',
  null
),
(
  'Another row, with JSON',
  '{ "a": 1, "b": ["dog", "cat", 42], "c": true }'::JSONB
);
"""

pg_curs.execute(create_table_statement)
pg_conn.commit()

In [0]:
pg_curs.execute(insert_statement)
pg_conn.commit()

In [28]:
query = "SELECT * FROM test_table;"
pg_curs.execute(query)
pg_curs.fetchall()

[(1, 'A row name', None),
 (2, 'Another row, with JSON', {'a': 1, 'b': ['dog', 'cat', 42], 'c': True})]

### ETL - RPG data from SQLite to PostgreSQL

In [29]:
!ls

rpg_db.sqlite3	sample_data


In [0]:
import sqlite3

sl_conn = sqlite3.connect('/content/rpg_db.sqlite3')
sl_curs = sl_conn.cursor()

In [31]:
# We care about charactercreator_character table

row_count = 'SELECT COUNT(*) FROM  charactercreator_character'
sl_curs.execute(row_count).fetchall()

[(302,)]

In [33]:
# Our goal = copy characters from sqlite to postgresql using python

# Step 1 Extract and get the characters

get_characters = 'SELECT * FROM charactercreator_character'
characters = sl_curs.execute(get_characters).fetchall()
characters[:5]

[(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1),
 (2, 'Optio dolorem ex a', 0, 0, 10, 1, 1, 1, 1),
 (3, 'Minus c', 0, 0, 10, 1, 1, 1, 1),
 (4, 'Sit ut repr', 0, 0, 10, 1, 1, 1, 1),
 (5, 'At id recusandae expl', 0, 0, 10, 1, 1, 1, 1)]

In [34]:
len(characters)

302

In [40]:
# Step 2 Transform

sl_curs.execute('PRAGMA table_info(charactercreator_character);').fetchall()

[(0, 'character_id', 'integer', 1, None, 1),
 (1, 'name', 'varchar(30)', 1, None, 0),
 (2, 'level', 'integer', 1, None, 0),
 (3, 'exp', 'integer', 1, None, 0),
 (4, 'hp', 'integer', 1, None, 0),
 (5, 'strength', 'integer', 1, None, 0),
 (6, 'intelligence', 'integer', 1, None, 0),
 (7, 'dexterity', 'integer', 1, None, 0),
 (8, 'wisdom', 'integer', 1, None, 0)]

In [0]:
create_character_table = """
CREATE TABLE charactercreator_character ( 
  character_id SERIAL PRIMARY KEY,
  name VARCHAR(30),
  level INT,
  exp INT,
  hp INT,
  strength INT,
  intelligence INT,
  dexterity INT,
  wisdom INT
);
"""


In [58]:
pg_curs.execute(create_character_table)
pg_conn.commit()

InFailedSqlTransaction: ignored

In [59]:
show_tables = """
SELECT
 *
FROM
  pg_catalog.pg_tables
WHERE
  schemaname != 'pg_catalog'
AND schemaname != 'information_schema';
"""

pg_curs.execute(show_tables)
pg_curs.fetchall()

InFailedSqlTransaction: ignored

In [52]:
characters[0]

(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

In [60]:
example_insert = """
INSERT INTO charactercreator_character
(name, level, exp, hp, strength, intelligence, dexterity, wisdom)
VALUES
""" + str(characters[0][1:]) + ";"

print(example_insert)


INSERT INTO charactercreator_character
(name, level, exp, hp, strength, intelligence, dexterity, wisdom)
VALUES
('Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1);


In [55]:
# How do we do this for all characters? Loops

for character in characters:
  insert_character = """
    INSERT INTO charactercreator_character
    (name, level, exp, hp, strength, intelligence, dexterity, wisdom)
    VALUES """ + str(character[1:]) + ";"
  pg_curs.execute(insert_character)

InFailedSqlTransaction: ignored

In [51]:
pg_curs.execute('SELECT * FROM charactercreator_character')
pg_curs.fetchall()

InFailedSqlTransaction: ignored